In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import altair as alt
from vega_datasets import data
import pandas as pd
import altair as alt
import socket

x = socket.gethostbyname("")


dataset = pd.read_csv('Sustainability_cleaned.csv', parse_dates=['Year'])

#dataset['Year'] = pd.to_datetime(dataset['Year'], format='%Y').dt.year

# Define the app and its layout
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


app.layout = dbc.Container([ 
    html.H1('World Sustainability Dashboard',
            style={
            'textAlign': 'center',
            'color': 'White',
            'background-color':'Blue',  
        }),
    dbc.Tabs([
        # First tab
        dbc.Tab(label='Environmental', children=[
            # First row
            dbc.Row([
                # First column (vertical slider)
                dbc.Col([
                    html.Label('Slider'),
                    dcc.Slider(
                        id='slider-env',
                        min=dataset['Year'].dt.year.min(),
                        max=dataset['Year'].dt.year.max(),
                        vertical= True,
                        step=1,
                        value=[min,max],
                        marks={str(year): str(year) for year in dataset['Year'].dt.year.unique()},
                    ),
                    dcc.Dropdown(
                        id='dropdown-env',
                        options=[{'label': i, 'value': i} for i in dataset['Country'].unique()],
                        value='Finland'
                    )],
                    width=3,md=3),
                # Second column (chart-1)
                dbc.Col(
                    html.Iframe(
                        id='env-1',
                        style={'border-width': '0', 'width': '100%', 'height': '400px'})),
                # Third column (chart-2)
                dbc.Col(
                    html.Iframe(
                        id='env-2',
                        style={'border-width': '0', 'width': '100%', 'height': '400px'}))     
            ]),
        ]),
    ]),# Tabs close
])# Container close


# Define the callbacks to update the charts




@app.callback(
    Output('env-1', 'srcDoc'),
    Input('slider-env', 'value'),
    Input('dropdown-env', 'value'))
def env_1(slider_value, dropdown_value):
    min_value = slider_value[0]
    max_value = slider_value[1]
    
    # Filter the data based on the dropdown values
    filtered_country = dataset[dataset['Country']== dropdown_value]


    # Filter the data based on the slider values
    filtered_year = filtered_country[(filtered_country['Year'] >= min_value) & (filtered_country['Year'] <= max_value)]
    
    # Create chart
    chart = alt.Chart(filtered_year).mark_circle(opacity=0.7, size=500).encode(
        alt.X('Internet', title='Access to Internet (% of population)'),
        alt.Y('Electricity_access', scale=alt.Scale(zero=False), title='Access to electricity (% of population)'),
        color=alt.Color('Income_classification', title='Income Classification'),
        tooltip=[alt.Tooltip('Internet', title='Access to Internet(% of population)'), 
                alt.Tooltip('Electricity_access',title='Access to electricity (% of population)')]).properties(
        width=500,
        height=400,
        title='Access to Utilities, as a percentage of Population'
    ).interactive()

    return chart.to_html()

@app.callback(
    Output('env-2', 'srcDoc'),
    Input('slider-env', 'value'),
    Input('dropdown-env', 'value'))
def env_2(slider_value, dropdown_value):
    min_value = slider_value[0]
    max_value = slider_value[1]
    
    # Filter the data based on the dropdown values
    filtered_country = dataset[dataset['Country']== dropdown_value]
    
    # Filter the data based on the slider values
    filtered_year = filtered_country[(filtered_country['Year'] >= min_value) & (filtered_country['Year'] <= max_value)]
    
    # create bar chart

    bar_chart = alt.Chart(filtered_year).mark_bar(color='Teal', size=10).encode(
        alt.X('Year:T',title =''),
        alt.Y('Co2_prod_tonnes:Q',title='Annual CO2 production (in tonnes)'),
        tooltip=[alt.Tooltip('Year:T', format='%Y'), alt.Tooltip('Co2_prod_tonnes:Q',title='Annual CO2 production')]
    )

    # create line chart


    line_chart = alt.Chart(filtered_year).mark_line(color='red').encode(
        alt.X('Year:T', title=''),
        alt.Y('Adj_savCO2_damage:Q', title='Adjusted Savings : CO2 Damage (% of GNI)'),
        tooltip=[alt.Tooltip('Year:T', format='%Y'), alt.Tooltip('Adj_savCO2_damage:Q',format='.2f',title='CO2 Damage (% of GNI)')]
    )
    
    # Combine line and bar chart
    
    chart = (bar_chart + line_chart).resolve_scale(
        y='independent'
    ).properties(
        width=500,
        height=400
    ).configure_axis(
        grid=False
    ).configure_view(
        strokeWidth=0
    )
   
    return chart.to_html()


if __name__ == '__main__':
    app.run_server(x)                  
                     
                     

